# Day 10: Syntax Scoring
The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

In [ ]:
let sample = "[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]"

let rec interpret (line:seq<char>) openBuffer =
    if Seq.isEmpty line then 0
    else
        let h = Seq.head line
        match h with
        //  If it's any of the openers, add our head to the top of the queue and move deeper
        | '('|'['|'{'|'<' -> seq { yield h; yield! openBuffer } |> interpret (Seq.tail line)
        //  If it's any of our closers in which the head of our buffer is their cohort, unwrap a bit and move deeper
        | ')' when Seq.head openBuffer = '(' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        | ']' when Seq.head openBuffer = '[' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        | '}' when Seq.head openBuffer = '{' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        | '>' when Seq.head openBuffer = '<' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        //  Line is corrupt
        | ')' -> 3
        | ']' -> 57
        | '}' -> 1197
        | '>' -> 25137
        | _ -> 0

let inline runEval (line:seq<char>) = interpret line []

// sample.Split(Environment.NewLine)
System.IO.File.ReadAllLines("../Data/Day10.txt") 
|> Seq.map runEval |> Seq.sum


364389

Now, discard the corrupted lines. The remaining lines are incomplete.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line. To repair the navigation subsystem, you just need to figure out the sequence of closing characters that complete all open chunks in the line.

In [ ]:
let inline calculateClosure buffer =
    let inline evalChar c = match c with '(' -> Some(1L) | '[' -> Some(2L) | '{' -> Some(3L) | '<' -> Some(4L) | _ -> None
    buffer |> Seq.choose evalChar |> Seq.fold (fun a i -> (a*5L)+i) 0L

let rec interpret (line:seq<char>) openBuffer =
    if Seq.isEmpty line then
        Some(calculateClosure openBuffer) 
    else
        let h = Seq.head line
        match h with
        //  If it's any of the openers, add our head to the top of the queue and move deeper
        | '('|'['|'{'|'<' -> seq { yield h; yield! openBuffer } |> interpret (Seq.tail line)
        //  If it's any of our closers in which the head of our buffer is their cohort, unwrap a bit and move deeper
        | ')' when Seq.head openBuffer = '(' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        | ']' when Seq.head openBuffer = '[' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        | '}' when Seq.head openBuffer = '{' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        | '>' when Seq.head openBuffer = '<' -> Seq.tail openBuffer |> interpret (Seq.tail line)
        //  Line is corrupt
        | _ -> None

let inline runEval (line:seq<char>) = interpret line []

// sample.Split(Environment.NewLine) 
System.IO.File.ReadAllLines("../Data/Day10.txt")
|> Array.choose runEval
|> Array.sort
|> Array.splitInto 2
|> Array.head
|> Array.last

2870201088